# Keras Working Example

### plaidml is to use 2018 MBP video card

In [8]:
# Install the plaidml backend
## DO THIS BEFORE IMPORTING KERAS OR TENSOR TO USE PLAIDML
import plaidml.keras
plaidml.keras.install_backend()

# Help MacOS be able to use Keras
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
# Gets rid of the processor warning.
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [9]:
## Use this as callback on .fit (callbacks=[cb])
## NOT necessary

# better progress bars for jupyter and won't kill model fit
# Command line the following line to get tqdm installed properly
# python -m pip install tqdm
from keras_tqdm import TQDMNotebookCallback
# keras, model definition...
cb = TQDMNotebookCallback()
setattr(cb,'on_train_batch_begin',lambda x,y:None)
setattr(cb,'on_train_batch_end',lambda x,y:None)

In [10]:
from __future__ import print_function
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import time
import glob
import cv2
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10

### Constants

In [11]:
batch_size = 32
num_classes = 10
epochs = 20
data_augmentation = True
num_predictions = 20
save_dir = os.path.join('../data/')
model_name = 'keras_cifar10_trained_model.h5'

### Use Keras provided data

In [13]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [14]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [15]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

INFO:plaidml:Opening device "metal_amd_radeon_pro_vega_20.0"


In [16]:
# This is equivalent to scaling
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [17]:
# set data_augmentation to True or False in the constants cell

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=12,
                        use_multiprocessing=True)

Using real-time data augmentation.
Epoch 1/20
1563/1563 [==============================] - 60s 38ms/step - loss: 1.8642 - acc: 0.3136 - val_loss: 1.5233 - val_acc: 0.4451
Epoch 2/20
1563/1563 [==============================] - 56s 36ms/step - loss: 1.5650 - acc: 0.4277 - val_loss: 1.3822 - val_acc: 0.4992
Epoch 3/20
1563/1563 [==============================] - 56s 36ms/step - loss: 1.4457 - acc: 0.4759 - val_loss: 1.2697 - val_acc: 0.5509
Epoch 4/20
1563/1563 [==============================] - 56s 36ms/step - loss: 1.3613 - acc: 0.5123 - val_loss: 1.2127 - val_acc: 0.5650
Epoch 5/20
1563/1563 [==============================] - 56s 36ms/step - loss: 1.2921 - acc: 0.5399 - val_loss: 1.1988 - val_acc: 0.5805
Epoch 6/20
1563/1563 [==============================] - 56s 36ms/step - loss: 1.2454 - acc: 0.5561 - val_loss: 1.0851 - val_acc: 0.6159
Epoch 7/20
1563/1563 [==============================] - 56s 36ms/step - loss: 1.1908 - acc: 0.5759 - val_loss: 1.0644 - val_acc: 0.6297
Epoch 8/20
15

In [18]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at ../data/keras_cifar10_trained_model.h5 


In [19]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 2s 190us/step
Test loss: 0.791367405796051
Test accuracy: 0.7241
